In [32]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
import spacy


In [33]:
df = pd.read_csv('data/cannabis.csv')

In [34]:
print(df.shape)
df


(2351, 6)


,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...,...
2346,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2347,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2348,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2349,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


In [36]:
df['Effects'].isnull().sum()

0

In [37]:
df[df['Description'].isnull()]

,Strain,Type,Rating,Effects,Flavor,Description
163,Banana-Candy,indica,4.2,"Relaxed,Euphoric,Uplifted,Creative,Happy","Tree,Fruit,Earthy,Sweet",NaN
368,Boysenberry,hybrid,3.6,"Giggly,Relaxed,Talkative,Energetic,Euphoric","Berry,Sweet,Earthy",NaN
594,Crater-Lake,hybrid,4.6,"Happy,Uplifted,Talkative,Relaxed,Creative","Citrus,Pine,Pungent",NaN
679,Domino,indica,4.0,"Sleepy,Happy,Tingly,Uplifted,Euphoric",NaN,NaN
736,Earthquake,hybrid,4.5,"Creative,Euphoric,Uplifted,Happy,Energetic","Woody,Pine,Earthy",NaN
817,Frosty,indica,4.4,"Sleepy,Relaxed,Creative,Hungry,Happy","Earthy,Sweet,Skunk",NaN
856,Ghost-Bubba,hybrid,4.6,"Happy,Energetic,Uplifted,Relaxed,Giggly","Sweet,Pungent,Grape",NaN
899,Goldwing,sativa,4.7,"Uplifted,Happy,Euphoric,Creative,Energetic",NaN,NaN
949,Green-Hornet,hybrid,4.2,"Euphoric,Uplifted,Relaxed,Giggly,Happy","Sweet,Woody,Pine",NaN
955,Green-Monster,hybrid,3.7,"Happy,Relaxed,Uplifted,Hungry,Giggly","Earthy,Chemical,Tar",NaN


In [41]:
df['Description'].isnull().value_counts()

False    2277
Name: Description, dtype: int64

In [42]:
df = df.dropna()

In [43]:
len(df['Description'])

2277

In [44]:
df["Description"].dtype

dtype('O')

In [45]:
df['Description'] = df['Description'].apply(str).str.replace('.', ',')

In [46]:
df["Description"].dtype

dtype('O')

In [47]:
df["Effects"][0]

'Creative,Energetic,Tingly,Euphoric,Relaxed'

In [48]:
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_lg")
df.head()

# The Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Make the tokens
tokens = []
for txt in tokenizer.pipe(df['Description'], batch_size=500):
    txt_tokens = [token.text for token in txt]
    tokens.append(txt_tokens)
df['tokens'] = tokens
df['tokens'].head()

0    [$100, OG, is, a, 50/50, hybrid, strain, that,...
1    [The, ‘98, Aloha, White, Widow, is, an, especi...
2    [1024, is, a, sativa-dominant, hybrid, bred, i...
3    [13, Dawgs, is, a, hybrid, of, G13, and, Chemd...
4    [Also, known, as, Kosher, Tangie,, 24k, Gold, ...
Name: tokens, dtype: object

In [49]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [50]:

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                       ngram_range = (1,2),
                       max_features = 5000)

# Create a vocabulary and tf-idf score per document
dtm = tfidf.fit_transform(df['Description'])

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn = nn.fit(dtm)


# View Feature Matrix as DataFrame
print(dtm.shape)
dtm.head()

(2277, 5000)


,10,10 11,10 week,10 weeks,100,11,11 week,11 weeks,12,13,...,yielding,yields,yields following,yields high,yoda,yunnan,zest,zesty,zesty lemon,zombie
0,0.0,0.0,0.0,0.0,0.390831,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.455537,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# Query Using kneighbors 
nn.kneighbors([dtm.iloc[0]])

(array([[0., 1., 1., 1.]]), array([[   0, 1607, 1606, 1605]]))

In [52]:
df['Description'][0][:150]

'$100 OG is a 50/50 hybrid strain that packs a strong punch, The name supposedly refers to both its strength and high price when it first started showi'

In [53]:
df['Description'][338][:150]

'A hybrid cross of Blueberry and Haze, provides a sweet smoke and pleasant high, \xa0Pace yourself with this one, overdoing it can send you to dreamland,\xa0'

In [54]:
ideal = ["""
Creative,Energetic,Tingly,Euphoric,Relaxed
"""]

In [55]:
# Query the ideal job
new = tfidf.transform(ideal)
new

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [56]:
nn.kneighbors(new.todense())

(array([[1., 1., 1., 1.]]), array([[1698, 1606, 1605, 1607]]))

In [57]:
# most ideal strain
df['Description'][1698]

'Purple Kush is a pure indica strain that emerged from the Oakland area of California as the result of a Hindu Kush and Purple Afghani cross, Its aroma is subtle and earthy with sweet overtones typical of Kush varieties, Blissful, long-lasting euphoria blankets the mind while physical relaxation rids the body of pain, sleeplessness, and stress, Purple Kush will grow wide rather than tall, and will be ready for harvest following an 8 week flowering time,'

In [58]:
df['Strain'][1698]

'Purple-Kush'